# Homework 3, Question 1: Normail Distribution (Gaussian)

In [1]:
import numpy as np
import scipy.integrate as spi

**Part A) Numerical Verification**

In [2]:
# Define the integrand function
def integrand(*v, A, w):
    v = np.array(v)  # Convert tuple to array
    exponent = -0.5 * v.T @ A @ v + v.T @ w
    return np.exp(exponent)

# Function to compute both numerical integral and closed-form result
def verify_integral(A, w):
    N = len(w)

    # Ensure A is symmetric positive definite
    if not np.allclose(A, A.T) or np.any(np.linalg.eigvals(A) <= 0):
        raise ValueError("Matrix A must be symmetric positive definite.")

    # Compute closed-form expression
    A_inv = np.linalg.inv(A)
    determinant_factor = np.sqrt(np.linalg.det(A_inv) * (2 * np.pi)**N)
    exponential_factor = np.exp(0.5 * w.T @ A_inv @ w)
    rhs = determinant_factor * exponential_factor

    # Perform numerical integration
    bounds = [(-10, 10)] * N  # Set integration limits
    lhs, error = spi.nquad(lambda *v: integrand(*v, A=A, w=w), bounds)

    return lhs, rhs, error

**Part b) Test Run**

In [3]:
w = np.array([1, 2, 3])
A = np.array([[4, 2, 1], [2, 5, 3], [1, 3, 6]])
A_prime = np.array([[4, 2, 1], [2, 1, 3], [1, 3, 6]])

In [4]:
# Run verification
lhs_A, rhs_A, error_A = verify_integral(A, w)

print("Numerical Integral (LHS):", lhs_A)
print("Closed-form Expression (RHS):", rhs_A)
print("Integration Error Estimate:", error_A)

Numerical Integral (LHS): 4.275823659012836
Closed-form Expression (RHS): 4.275823659011514
Integration Error Estimate: 2.525160790867566e-08


In [5]:
print((lhs_A - rhs_A)/error_A)

5.237281014802696e-05


The difference between the two expressions is less than the estimated error in this calculation. The evaluation works for A.

In [6]:
verify_integral(A_prime, w)

ValueError: Matrix A must be symmetric positive definite.

A works but A' doesn't because A' is not symmetric positive definite. It has negative eigenvalues. 

**Part c) Correlation function and moments of a multivariate normal distribution**

The expression for any $\langle v_i \rangle$ is:
$$\langle v_1 \rangle = \sum_{j=1}^{N} A^{-1}_{1j} w_j $$
where N is the dimension of the vector/matrix, in this case N=3. 

In [26]:
# Calculate the inverse of A
A_inv = np.linalg.inv(A)

# Compute the first moments ⟨v1⟩, ⟨v2⟩, ⟨v3⟩
moments = A_inv @ w

moments  # ⟨v1⟩, ⟨v2⟩, ⟨v3⟩

array([0.08955224, 0.10447761, 0.43283582])

$$\langle v_i v_j \rangle = A^{-1}_{ij} + \sum_{k=1}^{N} A^{-1}_{ik} w_k \sum_{m=1}^{N} A^{-1}_{jm} w_m$$

In [27]:
# Compute first moments ⟨v_i⟩ = sum_j A_inv[i, j] * w[j]
v_expectation = A_inv @ w

# Compute second moments ⟨v_i v_j⟩
def second_moment(i, j, A_inv, w):
    return A_inv[i, j] + v_expectation[i] * v_expectation[j]

# Compute specific values
v1v2 = second_moment(0, 1, A_inv, w)  # ⟨v1 v2⟩
v2v3 = second_moment(1, 2, A_inv, w)  # ⟨v2 v3⟩
v1v3 = second_moment(0, 2, A_inv, w)  # ⟨v1 v3⟩

print("⟨v1 v2⟩ =", v1v2)
print("⟨v2 v3⟩ =", v2v3)
print("⟨v1 v3⟩ =", v1v3)

⟨v1 v2⟩ = -0.12497215415460013
⟨v2 v3⟩ = -0.1040320784139006
⟨v1 v3⟩ = 0.0536867899309423


$$\langle v_1^2 v_2 \rangle = \sum_{a,b,c} A^{-1}_{1a} A^{-1}_{1b} A^{-1}_{2c} w_a w_b w_c + 2 A^{-1}_{11} \sum_c A^{-1}_{2c} w_c + A^{-1}_{12} \sum_b A^{-1}_{1b} w_b$$

In [28]:
# Compute third moments ⟨v_i^2 v_j⟩
def third_moment(i, j, A_inv, w):
    N = len(w)
    term1 = sum(A_inv[i, a] * A_inv[i, b] * A_inv[j, c] * w[a] * w[b] * w[c] 
                for a in range(N) for b in range(N) for c in range(N))
    term2 = 2 * A_inv[i, i] * sum(A_inv[j, c] * w[c] for c in range(N))
    term3 = A_inv[i, j] * sum(A_inv[i, b] * w[b] for b in range(N))
    return term1 + term2 + term3

# Compute specific values
v1sqv2 = third_moment(0, 1, A_inv, w)  # ⟨v1^2 v2⟩
v2v3sq = third_moment(1, 2, A_inv, w)  # ⟨v2 v3^2⟩

print("⟨v1^2 v2⟩ =", v1sqv2)
print("⟨v2 v3^2⟩ =", v2v3sq)

⟨v1^2 v2⟩ = 0.05430189218753642
⟨v2 v3^2⟩ = 0.2863018389895034


$$\langle v_1^2 v_2^2 \rangle = \sum_{a,b,c,d} A^{-1}_{1a} A^{-1}_{1b} A^{-1}_{2c} A^{-1}_{2d} w_a w_b w_c w_d + 2 A^{-1}_{11} \sum_{c, d} A^{-1}_{2c} A^{-1}_{2d} w_c w_d + 2 A^{-1}_{22} \sum_{a,b} A^{-1}_{1a} A^{-1}_{1b} w_a w_b + 4 A^{-1}_{12} \sum_{a,c} A^{-1}_{1a} A^{-1}_{2c} w_a w_c + 2 A^{-1}_{12} $$

In [ ]:
# Compute fourth moments ⟨v_i^2 v_j^2⟩
def fourth_moment(i, j, A_inv, w):
    N = len(w)
    term1 = sum(A_inv[i, a] * A_inv[i, b] * A_inv[j, c] * A_inv[j, d] * w[a] * w[b] * w[c] * w[d] 
                for a in range(N) for b in range(N) for c in range(N) for d in range(N))
    term2 = 2 * A_inv[i, i] * sum(A_inv[j, c] * A_inv[j, d] * w[c] * w[d] for c in range(N) for d in range(N))
    term3 = 2 * A_inv[j, j] * sum(A_inv[i, a] * A_inv[i, b] * w[a] * w[b] for a in range(N) for b in range(N))
    term4 = 4 * A_inv[i, j] * sum(A_inv[i, a] * A_inv[j, c] * w[a] * w[c] for a in range(N) for c in range(N))
    term5 = 2 * A_inv[i, j]**2
    return term1 + term2 + term3 + term4 + term5

# Compute specific values
v1sqv2sq = fourth_moment(0, 1, A_inv, w)  # ⟨v1^2 v2^2⟩
v2sqv3sq = fourth_moment(1, 2, A_inv, w)  # ⟨v2^2 v3^2⟩

print("⟨v1^2 v2^2⟩ =", v1sqv2sq)
print("⟨v2^2 v3^2⟩ =", v2sqv3sq)

⟨v1^2 v2^2⟩ = 0.04349713348453424
⟨v2^2 v3^2⟩ = 0.153439950065309
